In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.1 MB/s eta 0:00:00


In [3]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.4 MB/s eta 0:00:00


In [2]:
!pip install pyngrok

In [1]:
%%writefile app.py

import streamlit as st
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"

adapter_id = "mridul161203/qwen-lora"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )
tokenizer = AutoTokenizer.from_pretrained(model_name)

st.set_page_config(page_title="💬 Chatbot")

@st.cache_resource()
def ChatModel():
    model = AutoModelForCausalLM.from_pretrained(
        'unsloth/Qwen2.5-7B-Instruct-bnb-4bit',
        torch_dtype="auto",
        device_map="auto"
      )

    model.load_adapter(adapter_id)

    return model

with st.sidebar:
    st.title('💬 Chatbot')

    st.subheader('Models and parameters')

    temperature = st.sidebar.slider('temperature', min_value=0.01, max_value=2.0, value=0.1, step=0.01)
    top_p = st.sidebar.slider('top_p', min_value=0.01, max_value=1.0, value=0.9, step=0.01)
    max_length = st.sidebar.slider('max_length', min_value=64, max_value=4096, value=512, step=8)
    model = ChatModel()

if "messages" not in st.session_state.keys():
    st.session_state.messages = [{"role": "assistant", "content": "How may I assist you today?"}]

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

def clear_chat_history():
    st.session_state.messages = [{"role": "assistant", "content": "How may I assist you today?"}]
st.sidebar.button('Clear Chat History', on_click=clear_chat_history)

def format_data(system, prompt):
    return {"messages": [
                {
                    "role": "system",
                    "content": system
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        }

def generate_llama2_response(prompt_input):

    string_dialogue = "You are a helpful assistant. You do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant'."

    data = format_data(string_dialogue, prompt_input)

    messages = data["messages"]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        top_p=top_p,
        temperature=temperature,
        max_new_tokens=max_length
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

if prompt := st.chat_input():
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.write(prompt)

if st.session_state.messages[-1]["role"] != "assistant":
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = generate_llama2_response(prompt)
            placeholder = st.empty()
            full_response = ''
            for item in response:
                full_response += item
                placeholder.markdown(full_response)
            placeholder.markdown(full_response)
    message = {"role": "assistant", "content": full_response}
    st.session_state.messages.append(message)

Overwriting app.py


In [2]:
!streamlit run app.py &>/content/logs.txt &

In [3]:
from pyngrok import ngrok

ngrok.kill()

NGROK_AUTH_TOKEN = "$authtoken"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr="8501", proto="http", bind_tls=True)
print("streamlit UI:", ngrok_tunnel.public_url)

streamlit UI: https://b311-34-32-177-35.ngrok-free.app
